# Treniranje jezičkog modela

In [1]:
import flair.datasets
from flair.data import Sentence, Dictionary
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger, LanguageModel
from flair.trainers import ModelTrainer, LanguageModelTrainer, TextCorpus
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import collections
import glob
import pickle

### Generisanje rečnika

In [2]:
char_dictionary: Dictionary = Dictionary()
counter = collections.Counter()

In [3]:
files = glob.glob('corpus/corpus3/**/*', recursive=True)
files.remove('corpus/corpus3\\train')
print(files)

['corpus/corpus3\\test.txt', 'corpus/corpus3\\valid.txt', 'corpus/corpus3\\train\\train_split_1', 'corpus/corpus3\\train\\train_split_2']


In [4]:
# Broj procesuiranih linija
processed = 0

for file in files:

    with open(file, 'r', encoding='utf-8') as f:

        tokens = 0

        for line in f:
            processed += 1
            chars = list(line)
            tokens += len(chars)

            # Dodaj karaktere u rečnik
            counter.update(chars)


In [5]:
counter.keys()

dict_keys(['1', '9', '8', '4', 'K', 'a', 'o', ' ', 'i', 'b', 'č', 'n', ',', 'l', 'c', 'e', 'E', 'm', 'u', 'G', 'd', 'š', 't', 'j', 'N', 'r', 'g', 'p', '-', 's', 'k', '.', 'T', 'z', 'S', 'ž', 'h', 'đ', 'v', '(', ')', 'ć', 'P', 'V', 'B', 'D', 'O', ':', '–', 'f', '\n', '2', '0', 'I', 'M', 'A', 'U', 'L', 'Č', '’', '„', '!', '“', 'R', ';', 'J', 'Ž', 'Š', '3', 'Z', 'Ć', '5', '6', '7', '?', '/', 'H', '*', 'w', 'y', 'X', 'Y', 'ñ', '\xad', 'F', '=', 'â', '|', '`', '~'])

In [6]:
# Nabrojavanje elemenata sortirani po frekvenciji u korpusu i sumarizacija njihovih pojavljivanja
total_count = 0
for letter, count in counter.most_common():
    total_count += count

In [7]:
print("Total number of characters:",total_count)
print("Total number of processed lines:", processed)

Total number of characters: 52714
Total number of processed lines: 39


In [8]:
# Pomoćne strukture za generisanje rečnika
latin = set("abcčćddžđefghijklljmnnjoprsštuvzžABCČĆDDŽĐEFGHIJKLLJMNNJOPRSŠTUVZŽ")
numbers = set("0123456789")
punct = set(".?!,:;\"\'\/()[]{}_+-*^%#<>|&`~")

In [9]:
# Provere validnosti podskupova
latin.issubset(set(counter.keys()))

False

In [10]:
numbers.issubset(set(counter.keys()))

True

In [11]:
punct.issubset(set(counter.keys()))

False

In [12]:
summ = 0
idx = 0
for letter, count in counter.most_common():
    summ += count
    percentile = (summ / total_count)

    char_dictionary.add_item(letter)
    idx += 1
    print('%d\t%s\t%7d\t%7d\t%f' % (idx, letter, count, summ, percentile))

1	 	   9587	   9587	0.181868
2	a	   4680	  14267	0.270649
3	o	   4178	  18445	0.349907
4	e	   3831	  22276	0.422582
5	i	   3653	  25929	0.491881
6	n	   2613	  28542	0.541450
7	t	   2028	  30570	0.579922
8	s	   1929	  32499	0.616516
9	r	   1847	  34346	0.651554
10	j	   1725	  36071	0.684277
11	u	   1647	  37718	0.715521
12	d	   1574	  39292	0.745381
13	l	   1458	  40750	0.773039
14	m	   1442	  42192	0.800395
15	v	   1382	  43574	0.826612
16	k	   1332	  44906	0.851880
17	p	   1111	  46017	0.872956
18	g	    702	  46719	0.886273
19	,	    662	  47381	0.898831
20	b	    619	  48000	0.910574
21	z	    619	  48619	0.922317
22	.	    416	  49035	0.930208
23	č	    392	  49427	0.937645
24	š	    389	  49816	0.945024
25	ž	    272	  50088	0.950184
26	c	    269	  50357	0.955287
27	h	    218	  50575	0.959423
28	ć	    200	  50775	0.963217
29	-	    117	  50892	0.965436
30	đ	     83	  50975	0.967011
31	A	     83	  51058	0.968585
32	O	     79	  51137	0.970084
33	f	     75	  51212	0.971507
34	–	     74	  5128

In [13]:
with open('resources/char_mappings/latin_dict', 'wb') as f:
    mappings = {
        'idx2item': char_dictionary.idx2item,
        'item2idx': char_dictionary.item2idx
    }
    pickle.dump(mappings, f)

### Traniranje embedding-a

In [14]:
is_forward_lm = True
dictionary: Dictionary = Dictionary.load_from_file('resources/char_mappings/latin_dict')

# get your corpus, process forward and at the character level
corpus = TextCorpus('corpus/corpus3',
                    dictionary,
                    is_forward_lm,
                    character_level=True)

# instantiate your language model, set hidden size and number of layers
# U pozadini LanguageModel-a je rekurzivna neuronska mreza iz PyTorch bibloteke, konkretno LSTM ili GRU
language_model = LanguageModel(dictionary,
                               is_forward_lm,
                               hidden_size=128,
                               nlayers=1)
trainer = LanguageModelTrainer(language_model, corpus)

trainer.train('resources/taggers/language_model',
              sequence_length=50,
              mini_batch_size=100,
              max_epochs=128)

2022-08-30 15:56:14,883 read text file with 8 lines
2022-08-30 15:56:14,894 read text file with 8 lines
2022-08-30 15:56:17,005 Sequence length is 50
2022-08-30 15:56:17,006 Split 1	 - (15:56:17)
2022-08-30 15:56:17,388 best split so far
2022-08-30 15:56:17,389 best loss so far 3.85702619
2022-08-30 15:56:17,634 ('\n!(asm 3!  6Č0šoa6, GHv u mm d/4F 9u š fl 4 XYeŠ .\ny9jU.V â !    (J    š l   Ng  šv )Yt čd š -X ~v !r* u Lmss  e  K “! D e n-v Fs r  č   i R; G   điu\n  Pš s    S      J 4Ć |JlN  Y  .m    9J  V - A3Pd  ,ž  ! Z j -9O3Ji  R K “     m  G         Gš– .o1Nđ w:B     9âiL â  Y I it8ut,Jo    u   V:        0 va~pI     | Isr ~   ?=* kD   c / S  v  G ;lŽG  4  Fv s     Fi      –o   ž 0 mid„ o   m  w:F0’~)i    maU r   „*     Gu b yM č \niKr  <unk>e d    o-už  iž `` I7     -   kv Ai’5! M o  k  YO0r H1čđČ9Be *          9l*. L4UI –f B X     ć R  Ć  Ži   PR   UĆ  to  YJ i GâajoE   i  vB   \ni  n  * Z=oF     \xad’  w cŠ  X`9  i 2o  wM  k    Z (og oX~  v9uMz5ai tu kFj3 a â\xad a6rPâ   sJm“ m 

In [15]:
sentence = Sentence('Ja volim Beograd.')

# init embeddings from your trained LM
char_lm_embeddings = FlairEmbeddings('resources/taggers/language_model/best-lm.pt')

# embed sentence
char_lm_embeddings.embed(sentence)

for token in sentence:
    print(token)
    print(token.embedding)

Token[0]: "Ja"
tensor([ 7.8041e-02,  6.3421e-02, -1.3366e-01,  1.9769e-01, -5.4676e-01,
        -4.2812e-01, -4.8944e-02, -2.1615e-02, -7.5816e-02,  1.6148e-02,
         1.3920e-01,  1.0232e-01, -4.2299e-03,  3.0809e-01,  6.6801e-03,
        -5.6751e-02, -5.0063e-02, -1.3072e-02, -2.4533e-01,  4.8261e-02,
        -6.7669e-01, -1.1872e-01,  8.0200e-02,  2.2822e-03, -9.9832e-03,
         9.6121e-02, -3.3470e-01, -3.0448e-02,  6.1459e-01,  5.3325e-02,
         1.0001e-02,  2.2809e-01, -1.8127e-02, -7.1859e-02,  1.3836e-03,
        -3.2170e-02, -1.1790e-01,  4.4122e-02, -2.3794e-01,  1.1977e-02,
         9.7364e-02, -1.1389e-01, -5.0077e-02, -7.3163e-01,  5.7625e-01,
        -1.4826e-01, -1.6338e-02,  3.2111e-02, -4.2402e-02, -6.7450e-03,
         1.5009e-02, -1.8348e-02,  1.0191e-03, -1.0547e-01,  2.9936e-02,
         8.1123e-03, -3.6458e-02, -2.4391e-02,  5.9031e-01, -2.4301e-01,
         2.8506e-01,  3.3756e-01,  2.2274e-01, -1.0784e-03,  1.1249e-01,
        -3.8228e-02, -3.4465e-02, -1

### Podela postojećeg korpusa na dva dela
Koristili smo kao inicijalni korpus UD_SERBIAN, koji je podeljen po svojoj strukturi na train, test i dev skupove.
Treniranje SequenceTagger koristi validation skup korpusa, pa je bilo potrebno da dev skup podelimo na 2 dela:
1. Jedan će koristiti SequenceTagger za internu validaciju
2. Drugi ćemo mi koristiti za validaciju hiperparametara po kojima optimizujemo postojeći model

Sličnu stvar smo uradili i sa test skupom, iz istog raloga.

In [16]:
# define columns
columns = {0: 'id', 1: 'text', 2: 'ner', 3: 'upos'}

# this is the folder in which train, test and dev files reside
data_folder = 'corpus/corpus1'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus1 = ColumnCorpus(data_folder, columns,
                              train_file='sr_set-ud-train.conllu',
                              test_file='sr_set-ud-test2.conllu',
                              dev_file='sr_set-ud-dev2.conllu')


2022-08-30 16:03:11,819 Reading data from corpus\corpus1
2022-08-30 16:03:11,820 Train: corpus\corpus1\sr_set-ud-train.conllu
2022-08-30 16:03:11,820 Dev: corpus\corpus1\sr_set-ud-dev2.conllu
2022-08-30 16:03:11,821 Test: corpus\corpus1\sr_set-ud-test2.conllu


In [17]:
# define columns
columns = {0: 'id', 1: 'text', 2: 'ner', 3: 'upos'}

# this is the folder in which train, test and dev files reside
data_folder = 'corpus/corpus2'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus2 = ColumnCorpus(data_folder, columns,
                              train_file='sr_set-ud-train.conllu',
                              test_file='sr_set-ud-test.conllu',
                              dev_file='sr_set-ud-dev.conllu')

2022-08-30 16:03:14,655 Reading data from corpus\corpus2
2022-08-30 16:03:14,656 Train: corpus\corpus2\sr_set-ud-train.conllu
2022-08-30 16:03:14,656 Dev: corpus\corpus2\sr_set-ud-dev.conllu
2022-08-30 16:03:14,657 Test: corpus\corpus2\sr_set-ud-test.conllu


In [18]:
label_type = 'upos'
label_dict  = corpus1.make_label_dictionary(label_type = label_type)

2022-08-30 16:03:17,244 Computing label dictionary. Progress:


3328it [00:00, 29981.84it/s]

2022-08-30 16:03:17,359 Dictionary created for label 'upos' with 18 values: NOUN (seen 18103 times), PUNCT (seen 9351 times), ADJ (seen 8835 times), ADP (seen 7130 times), VERB (seen 6406 times), PROPN (seen 5622 times), AUX (seen 4667 times), DET (seen 2848 times), SCONJ (seen 2713 times), ADV (seen 2543 times), CCONJ (seen 2541 times), PRON (seen 1859 times), NUM (seen 944 times), PART (seen 461 times), X (seen 232 times), INTJ (seen 3 times), SYM (seen 1 times)


### Primena istreniranog embedding-a
Iskoristićemo vektorske reprezentacije tokena iz embedding modela kog smo trenirali da bismo napravili model za predikciju UPOS tagova. Uvezali smo Character embedding model koji smo istrenirali sa vec postojecim word embedding modelom (koji funkcionise na nivou reci) i tu strukturu prosledili modelu za predkiciju UPOS tagova.

In [19]:
embedding_types = [
    WordEmbeddings('glove'),
    FlairEmbeddings('resources/taggers/language_model/best-lm.pt')
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

In [20]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

trainer = ModelTrainer(tagger, corpus1)

2022-08-30 16:03:20,701 SequenceTagger predicts: Dictionary with 18 tags: <unk>, NOUN, PUNCT, ADJ, ADP, VERB, PROPN, AUX, DET, SCONJ, ADV, CCONJ, PRON, NUM, PART, X, INTJ, SYM


In [21]:
best_score = 0.0
best_params = {'learning_rate':0, 'mini_batch_size': 0, 'max_epochs' : 10}

param_learning_rates = np.linspace(0.001, 0.6, num=4)
param_mini_batch_sizes = np.arange(10,100,20)
max_epochs = np.arange(10,13,1)

model_history = []

In [ ]:
for i,lr in enumerate(param_learning_rates):
    for j,mbs in enumerate(param_mini_batch_sizes):
        for k,me in enumerate(max_epochs):
                print('\n\n\n')
                print("#######################################################")
                print(f"################ MODEL NUMBER {i}, {j}, {k} #######################")
                print("#######################################################")
                trainer.train('resources/taggers/language_model_testing',
                              learning_rate=lr,
                              mini_batch_size=int(mbs),
                              max_epochs=int(me),
                              write_weights = True)
                model = SequenceTagger.load('resources/taggers/optimized-upos/final-model.pt')
                actual= np.array([])
                predicted = np.array([])

                # Validiramo na drugoj "polovini" validacionog skupa/corpusa
                # (onoj koja nije prosledjena ranijem traineru)
                for actualSentence in corpus2.dev:

                    for token in actualSentence:
                        actual = np.append(actual, token.get_label('upos').value)

                    predictedSentence = Sentence([token.text for token in actualSentence.tokens])
                    model.predict(predictedSentence)
                    for token in predictedSentence:
                        predictedLabels = token.get_labels('upos')
                        for predictedLabel in predictedLabels:
                            predicted = np.append(predicted, predictedLabel.value)
                score = metrics.accuracy_score(actual,predicted)

                if score>best_score:
                    best_params['learning_rate'] = lr
                    best_params['param_mini_batch_sizes'] = mbs
                    best_params['max_epochs'] = me

                report = metrics.classification_report(actual,predicted)
                params = {'learning_rate': lr, 'param_mini_batch_sizes': mbs,
                          'max_epochs': me}
                model_history.append({'params':params,'report':report})





#######################################################
################ MODEL NUMBER 1 #######################
#######################################################
2022-08-30 16:03:20,746 ----------------------------------------------------------------------------------------------------
2022-08-30 16:03:20,746 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(91, 100)
        (rnn): LSTM(100, 128)
        (decoder): Linear(in_features=128, out_features=91, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=228, out_features=228, bias=True)
  (rnn): LSTM(228, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512,

C:\Users\vladimir\anaconda3\envs\ML3\lib\site-packages\flair\trainers\trainer.py:64: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2022-08-30 16:03:25,731 epoch 1 - iter 33/333 - loss 3.55456821 - samples/sec: 66.36 - lr: 0.001000
2022-08-30 16:03:30,862 epoch 1 - iter 66/333 - loss 3.52915854 - samples/sec: 65.02 - lr: 0.001000
2022-08-30 16:03:35,218 epoch 1 - iter 99/333 - loss 3.49130203 - samples/sec: 76.69 - lr: 0.001000
2022-08-30 16:03:39,936 epoch 1 - iter 132/333 - loss 3.45640381 - samples/sec: 70.91 - lr: 0.001000
2022-08-30 16:03:45,535 epoch 1 - iter 165/333 - loss 3.41364441 - samples/sec: 59.56 - lr: 0.001000
2022-08-30 16:03:49,870 epoch 1 - iter 198/333 - loss 3.38043681 - samples/sec: 77.27 - lr: 0.001000
2022-08-30 16:03:54,841 epoch 1 - iter 231/333 - loss 3.34791162 - samples/sec: 67.17 - lr: 0.001000
2022-08-30 16:03:59,300 epoch 1 - iter 264/333 - loss 3.31396478 - samples/sec: 74.95 - lr: 0.001000
2022-08-30 16:04:04,312 epoch 1 - iter 297/333 - loss 3.28054511 - samples/sec: 66.64 - lr: 0.001000
2022-08-30 16:04:09,507 epoch 1 - iter 330/333 - loss 3.25244578 - samples/sec: 64.26 - lr: 0.

100%|██████████| 28/28 [00:01<00:00, 16.97it/s]

2022-08-30 16:04:11,692 Evaluating as a multi-label problem: False
2022-08-30 16:04:11,721 DEV : loss 2.917466402053833 - f1-score (micro avg)  0.091
2022-08-30 16:04:11,741 BAD EPOCHS (no improvement): 0
2022-08-30 16:04:11,742 saving best model


...

2022-08-31 00:21:01,795 Evaluating as a multi-label problem: False
2022-08-31 00:21:01,829 DEV : loss 0.38438165187835693 - f1-score (micro avg)  0.864
2022-08-31 00:21:01,847 BAD EPOCHS (no improvement): 3
2022-08-31 00:21:01,848 ----------------------------------------------------------------------------------------------------


2022-08-31 00:21:05,041 epoch 7 - iter 6/67 - loss 0.57476657 - samples/sec: 93.99 - lr: 0.600000
2022-08-31 00:21:08,447 epoch 7 - iter 12/67 - loss 0.57905932 - samples/sec: 89.63 - lr: 0.600000
2022-08-31 00:21:11,811 epoch 7 - iter 18/67 - loss 0.57191022 - samples/sec: 90.99 - lr: 0.600000
2022-08-31 00:21:14,883 epoch 7 - iter 24/67 - loss 0.57047345 - samples/sec: 99.63 - lr: 0.600000
2022-08-31 00:21:18,074 epoch 7 - iter 30/67 - loss 0.56838901 - samples/sec: 95.88 - lr: 0.600000
2022-08-31 00:21:21,125 epoch 7 - iter 36/67 - loss 0.56906419 - samples/sec: 100.30 - lr: 0.600000
2022-08-31 00:21:24,223 epoch 7 - iter 42/67 - loss 0.57109135 - samples/sec: 98.81 - lr: 0.600000
2022-08-31 00:21:27,683 epoch 7 - iter 48/67 - loss 0.56825945 - samples/sec: 88.18 - lr: 0.600000
2022-08-31 00:21:30,956 epoch 7 - iter 54/67 - loss 0.56886531 - samples/sec: 93.23 - lr: 0.600000
2022-08-31 00:21:34,226 epoch 7 - iter 60/67 - loss 0.56999744 - samples/sec: 93.57 - lr: 0.600000
2022-08-31

100%|██████████| 6/6 [00:00<00:00,  6.56it/s]

2022-08-31 00:21:38,962 Evaluating as a multi-label problem: False
2022-08-31 00:21:38,995 DEV : loss 0.3661198019981384 - f1-score (micro avg)  0.8717
2022-08-31 00:21:39,011 Epoch     7: reducing learning rate of group 0 to 3.0000e-01.
2022-08-31 00:21:39,012 BAD EPOCHS (no improvement): 4
2022-08-31 00:21:39,013 ----------------------------------------------------------------------------------------------------


2022-08-31 00:21:42,296 epoch 8 - iter 6/67 - loss 0.56305297 - samples/sec: 91.44 - lr: 0.300000
2022-08-31 00:21:45,635 epoch 8 - iter 12/67 - loss 0.55491812 - samples/sec: 91.52 - lr: 0.300000
2022-08-31 00:21:48,862 epoch 8 - iter 18/67 - loss 0.54469057 - samples/sec: 94.88 - lr: 0.300000
2022-08-31 00:21:52,094 epoch 8 - iter 24/67 - loss 0.54422080 - samples/sec: 94.79 - lr: 0.300000
2022-08-31 00:21:55,536 epoch 8 - iter 30/67 - loss 0.54466601 - samples/sec: 88.99 - lr: 0.300000
2022-08-31 00:21:58,951 epoch 8 - iter 36/67 - loss 0.53967275 - samples/sec: 89.71 - lr: 0.300000
2022-08-31 00:22:02,536 epoch 8 - iter 42/67 - loss 0.53730791 - samples/sec: 85.06 - lr: 0.300000
2022-08-31 00:22:05,894 epoch 8 - iter 48/67 - loss 0.54140711 - samples/sec: 91.24 - lr: 0.300000
2022-08-31 00:22:09,299 epoch 8 - iter 54/67 - loss 0.54125719 - samples/sec: 89.69 - lr: 0.300000


In [ ]:
# Treniranje modela koji ima optimalne vrednosti hiperparametara
trainer.train('resources/taggers/language_model_testing',
                              learning_rate=best_params['learning_rate'],
                              mini_batch_size=int(best_params['param_mini_batch_sizes']),
                              max_epochs=int(best_params['max_epochs']),
                              write_weights = True)

In [ ]:
# Učitavanje istreniranog modela
model = SequenceTagger.load('resources/taggers/language_model_testing/final-model.pt')

In [ ]:
actualSentenceExample = corpus2.dev[17]
predictedSentenceExample = Sentence(actualSentenceExample.to_plain_string())
model.predict(predictedSentenceExample)
print("\n\n")
# Predvidi etikete i štampaj

for i,token in enumerate(predictedSentenceExample):
    predictedLabels = token.get_labels('upos')
    actualLabels = actualSentenceExample[i].get_labels('upos')

    for j, label in enumerate(predictedLabels):
        token = label.data_point.form
        predictedValue = label.value
        actualValue = actualLabels[j].value

        print(token, "- predicted:" ,predictedValue, ", actual:", actualValue)

### Konstrukcija matrice konfuzije

In [ ]:
actual= np.array([])
predicted = np.array([])

for actualSentence in corpus2.dev:
    predictedSentence = Sentence(actualSentence.to_plain_string())
    model.predict(predictedSentence)


    numTokens = min(len(predictedSentence.tokens),len(actualSentence.tokens))
    i=0
    punctCount = 0

    while i+punctCount<numTokens:
        actualLabels = actualSentence[i].get_labels('upos')

        if predictedSentence[i+punctCount].labels[0].data_point.form == ".":
            punctCount+=1

        if i+punctCount >= numTokens:
            break
        predictedLabels = predictedSentence[i+punctCount].get_labels('upos')

        numLabels = min(len(actualLabels),len(predictedLabels))
        j=0
        while j<numLabels:
            predicted = np.append(predicted, predictedLabels[j].value)
            actual = np.append(actual, actualLabels[j].value)
            j+=1
        i+=1


In [ ]:
classes = np.union1d(np.unique(actual), np.unique(predicted))
cm = metrics.confusion_matrix(actual, predicted)
cm_df = pd.DataFrame(cm,
                     index = classes,
                     columns = classes)

# Crtanje
plt.figure(figsize=(15,13))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
sentence = Sentence('Srbija je tokom bombardovanja devedesetih izgubila mnoge kulturne znamenitosti.')
model.predict(sentence)

print(sentence.to_tagged_string())